# DaNLP Test

Test the DaNLP software for linguistic processing of Danish text, in particular the dependency parsing. Links:
    
* Installation: https://pypi.org/project/danlp/
* Dependency parsing: https://github.com/alexandrainst/danlp/blob/master/docs/docs/tasks/dependency.md

In [1]:
import danlp

In [3]:
from danlp.models import load_spacy_model

2022-01-21 11:40:49.355128: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-21 11:40:49.355167: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


AttributeError: module 'torch.jit' has no attribute '_script_if_tracing'